# Fine-tuning을 이용한 AI에게 인격 부여하기
## OpenAI Fine-tuning API로 학습하고 대화하기

In [ ]:
from openai import OpenAI

In [ ]:
client = OpenAI()

## Upload File

In [ ]:
train_file = client.files.create(
  file=open("train.jsonl", "rb"),
  purpose="fine-tune"
)

In [ ]:
train_file

In [ ]:
train_file.dict()

In [ ]:
train_file.id

In [ ]:
valid_file = client.files.create(
  file=open("valid.jsonl", "rb"),
  purpose="fine-tune"
)

In [ ]:
valid_file

In [ ]:
valid_file.dict()

In [ ]:
valid_file.id

## Finetuning

### Finetuning job 제출하기

In [ ]:
job = client.fine_tuning.jobs.create(
  training_file=train_file.id,
  validation_file=valid_file.id, 
  model="gpt-3.5-turbo-1106",
  hyperparameters={
    "n_epochs": 1 # default: 3
  }
)

In [ ]:
job

In [ ]:
job.dict()

In [ ]:
job.id

In [ ]:
print("Job ID:", job.id)
print("Status:", job.status)

### Finetuning job들 가져오기

In [ ]:
resp = client.fine_tuning.jobs.list(limit=10)

In [ ]:
resp.dict()

### 현재 Finetuning 상태 가져오기


In [ ]:
job = client.fine_tuning.jobs.retrieve(job.id)

In [ ]:
job.dict()

In [ ]:
print("Job ID:", job.id)
print("Status:", job.status)

### Finetuning job 캔슬하기

In [ ]:
# client.fine_tuning.jobs.cancel(job.id)

### 학습 과정 확인하기

In [ ]:
# List up to 10 events from a fine-tuning job
response = client.fine_tuning.jobs.list_events(fine_tuning_job_id=job.id, limit=10)
events = response.data
events.reverse()

for event in events:
    print(event.message)

### Finetuning 모델 삭제하기

In [ ]:
# Delete a fine-tuned model (must be an owner of the org the model was created in)
client.models.delete("ft:gpt-3.5-turbo:acemeco:suffix:abc123")

# Finetuning된 모델 Inference하기

In [3]:
import os
from openai import OpenAI
client = OpenAI()

In [4]:
system_prompt ="""\
- 너는 햄버거 가게의 직원이다.
- 아래의 단계로 질문을 한다.
1. 주문 할 메뉴 묻기
2. 더 주문 할 것이 없는지 묻기
3. 여기서 먹을지 가져가서 먹을지 질문한다.
4. 카드로 계산할지 현금으로 계산할지 질문한다.
4. 주문이 완료되면 인사를 하고 [END] 라고 이야기한다.
- 너는 영어로 답한다."""

In [4]:
model = "ft:gpt-3.5-turbo-1106:liam::8gr0Gkrs"

In [6]:
response = client.chat.completions.create(
  model=model,
  messages=[
    {"role": "system", "content": system_prompt},
    {"role": "user", "content": "안녕"}
  ]
)
print(response.choices[0].message.content)

긍정


In [7]:
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.schema import StrOutputParser

In [8]:
llm = ChatOpenAI(model=model)

In [9]:
prompt_template = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}" )
    ]
)

In [10]:
chain = prompt_template | llm | StrOutputParser()

In [12]:
chain.invoke({"input": "안녕"})

'부정'